In [1]:
import rqdatac
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import FamaMacBeth
import csv
import numpy as np
import import_ipynb
import a_general_factor_test
import b_micro_factor_loader_library

In [2]:
# main_script.py

import rqdatac
import pandas as pd
import a_general_factor_test
import b_micro_factor_loader_library

# --- 1. 初始化和定义参数 ---
rqdatac.init()
start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2020-12-31')
instrument_type = 'Convertible'
rolling_window = 90

# --- 2. 【核心优化】在所有循环开始前，获取一次公共数据 ---
print(">>> 正在获取公共数据（资产池和收益率），此操作仅执行一次...")
try:
    # 这两个函数只会被调用一次，极大节省了时间和API配额
    all_instruments = a_general_factor_test.get_bonds_poll(start_date, end_date, instrument_type)
    dependent_data = a_general_factor_test.get_return_rate(all_instruments, start_date, end_date)
    print(">>> 公共数据获取完毕。")
except Exception as e:
    print(f"获取公共数据时失败，程序终止。错误: {e}")
    exit() # 如果公共数据都拿不到，后续无法进行

# --- 3. 定义要测试的因子和分组 ---
maturity_groups = ['短期 (1-3年)', '中期 (3-5年)', '长期 (5+年)']
factor_list = ['momentum_5d', 'momentum_10d', 'momentum_20d','momentum_60d', 'momentum_90d',
               'volatility_20d','volatility_60d','volatility_90d','avg_volume_20d','avg_volume_60d',
               'avg_volume_90d','volume_shock','rsi_14d','macd']

# --- 4. 【新结构】循环测试 ---
for group in maturity_groups:
    for factor in factor_list:
        folder_name = f'micro_{group}_factor'
        print(f"\n--- 正在处理: 分组[{group}], 因子[{factor}] ---")
        
        # 您不再需要 main_test_for_micro_get_factors 这个“大而全”的函数了
        # 直接在循环里调用需要的逻辑
        try:
            # a) 生成因子暴露度（Beta值），这是每次循环唯一需要新计算的部分
            rolling_betas = b_micro_factor_loader_library.generate_micro_factors(all_instruments, start_date, end_date, factor)

            # b) 后续处理
            rolling_betas_tagged = a_general_factor_test.add_maturity_group_tag(rolling_betas, all_instruments)
            rolling_betas_selected = a_general_factor_test.select_wanted_rolling_data(rolling_betas_tagged, group)
            
            # c) F-M回归（使用预先获取的 dependent_data）
            result = a_general_factor_test.do_fama_macbeth(rolling_betas_selected, dependent_data) 
            
            # d) 结果存储
            success_path, dropped_path, error_path = a_general_factor_test.generate_file_path(folder_name)
            a_general_factor_test.store_value_into_file(result, factor, success_path, dropped_path, error_path)

        except Exception as e:
            print(f"处理因子 {factor} 在分组 {group} 时发生错误: {e}")
            # 可以在这里记录详细错误到日志文件

/opt/anaconda3/envs/rqsdk/lib/python3.9/site-packages/rqdatac/client.py:257: UserWarning: Your account will be expired after  28 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


>>> 正在获取公共数据（资产池和收益率），此操作仅执行一次...
testmodel2
可转债数量: 449
>>> 公共数据获取完毕。

--- 正在处理: 分组[短期 (1-3年)], 因子[momentum_5d] ---
开始获取基础量价数据...
数据获取完毕，开始计算因子...
正在计算动量因子...
正在计算波动率因子...
正在计算流动性因子...
正在计算技术指标 (RSI, MACD)...
因子计算完成，共生成 14 个微观因子。
model2
                             FamaMacBeth Estimation Summary                            
Dep. Variable:                     return_30   R-squared:                       -0.0014
Estimator:                       FamaMacBeth   R-squared (Between):             -0.0267
No. Observations:                      63833   R-squared (Within):               0.0010
Date:                       Fri, Jul 18 2025   R-squared (Overall):             -0.0014
Time:                               16:15:25   Log-likelihood                 5.627e+04
Cov. Estimator:    Fama-MacBeth Standard Cov                                           
                                               F-statistic:                     -91.814
Entities:                                254   P-value     